In [1]:
!pip install gtts
!pip install youtube_transcript_api
!pip install networkx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.3/622.3 kB 10.3 MB/s eta 0:00:00


In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
from nltk.tokenize import sent_tokenize
import numpy as np
import networkx as nx
import re

In [3]:
# Ensure NLTK resources are downloaded
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [4]:
# Function to read and split the input text into sentences
def read_article(text):
    sentences = sent_tokenize(text)  # Tokenize the input text into sentences
    for i in range(len(sentences)):
        sentences[i] = re.sub(r"[^a-zA-Z0-9]", " ", sentences[i])  # Remove non-alphanumeric characters
    return sentences

In [5]:
# Function to calculate sentence similarity based on cosine distance
def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []

    # Tokenize and clean the sentences by converting them to lowercase
    sent1 = [w.lower() for w in sent1.split() if w.lower() not in stopwords]
    sent2 = [w.lower() for w in sent2.split() if w.lower() not in stopwords]

    # Create a list of all unique words in both sentences
    all_words = list(set(sent1 + sent2))

    # Create vectors for each sentence, with word counts
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)

    # Build the vector for the first sentence
    for w in sent1:
        vector1[all_words.index(w)] += 1

    # Build the vector for the second sentence
    for w in sent2:
        vector2[all_words.index(w)] += 1

    # Return cosine distance (1 - cosine_distance) to make the similarity higher for more similar sentences
    return 1 - cosine_distance(vector1, vector2)

In [6]:
# Function to build a similarity matrix from the sentences
def build_similarity_matrix(sentences, stop_words):
    # Create an empty similarity matrix (2D array)
    similarity_matrix = np.zeros((len(sentences), len(sentences)))

    # Calculate the similarity between each pair of sentences
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 != idx2:  # Don't compare the sentence to itself
                similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)

    return similarity_matrix


In [7]:
# Main function to generate the summary
def generate_summary(text, top_n):
    # Get stop words from NLTK
    stop_words = stopwords.words('english')

    # Read the article and tokenize into sentences
    sentences = read_article(text)

    # Build the similarity matrix
    sentence_similarity_matrix = build_similarity_matrix(sentences, stop_words)

    # Create a similarity graph using NetworkX
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_matrix)

    # Rank sentences using PageRank algorithm
    scores = nx.pagerank(sentence_similarity_graph)

    # Sort sentences based on their rank scores
    ranked_sentences = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)

    # Select the top N sentences for the summary
    summarize_text = []
    for i in range(top_n):
        if i < len(ranked_sentences) and isinstance(ranked_sentences[i], (list, tuple)) and len(ranked_sentences[i]) > 1:
            summarize_text.append(ranked_sentences[i][1])

    # Return the summarized text and original sentence count
    return " ".join(summarize_text), len(sentences)

In [8]:
# Import the required module for text-to-speech conversion
from gtts import gTTS
import re
import os
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled, CouldNotRetrieveTranscript

In [9]:
def generate_transcript(video_id):
    try:
        # Fetch the transcript
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        # Calculate the total number of words
        no_of_words = sum(len(item['text'].split()) for item in transcript)

        # Combine transcript into a single string
        script = " ".join(item["text"] for item in transcript if item["text"] != '[Music]')
        return script, no_of_words

    except TranscriptsDisabled:
        print(f"Transcripts are disabled for the video {video_id}.")
        return None, 0

    except CouldNotRetrieveTranscript:
        print(f"Could not retrieve transcript for video {video_id}. The video might not have a transcript.")
        return None, 0

    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None, 0

In [19]:
# generate summary from video link
def generate_video_summary(video_id):
    #id = input('Enter the link of the video: ')
    video_id = video_id.split("=", 1)[1]

    # Fetch transcript
    transcript, no_of_words = generate_transcript(video_id)

    if transcript is None:
        p="No transcript available. Exiting."
        return p,p

    # Clean transcript text
    text = transcript.replace("♪", "")
    text = re.escape(text)  # Escape special characters
    #generate a transcript summary
    summary, original_length = generate_summary(text, top_n=5)
    return summary

#generate_video_summary("https://www.youtube.com/watch?v=tjM9IoszCvA")
#generate_video_summary("https://www.youtube.com/watch?v=dsto27EQQKM")


In [11]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 8.3 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [12]:
import gradio as gr

In [20]:
gr_interface = gr.Interface(fn=generate_video_summary,
                            inputs=gr.Textbox(lines=1, placeholder="Enter video id to summarize..."),
                            outputs=gr.Textbox(label="Abstractive Summary"))

# Launch the Gradio interface
gr_interface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1f7baa216f7b1b8e37.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [16]:
import os
from gtts import gTTS

Generating summary audio in required language

In [18]:
language = 'ta'
id=input("Enter the youtube video link:")
# Convert text to speech
text=generate_video_summary(id)
myobj = gTTS(text=text, lang=language, slow=False)

# Save the audio file
audio_file = "welcome.mp3"
myobj.save(audio_file)

# Play the audio file
os.system(f"mpg321 {audio_file}")

Enter the youtube video link:https://www.youtube.com/watch?v=dsto27EQQKM


32512